In [1]:
DATASET_DIR = "/content/drive/MyDrive/Work/Aj Rapeepan/MRI/Alzheimer's /Dataset/"

In [4]:
!pip install pennylane
!pip install --upgrade pennylane

In [3]:
import numpy as np
#load  numpy dataset
xs = np.load(DATASET_DIR + "preprocessed_xs.npy")
ys = np.load(DATASET_DIR + "ys.npy")
ys_encoded = np.load(DATASET_DIR + "ys_encoded.npy")

print("xs shape:", xs.shape)
print("ys shape:", ys.shape)
print("ys_encoded shape:", ys_encoded.shape)

xs shape: (6400, 256, 256, 3)
ys shape: (6400,)
ys_encoded shape: (6400,)


In [11]:
import os
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, EfficientNetV2S
from pennylane.qnn.keras import KerasLayer

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Hide TensorFlow warnings


class QENNAClassifier:
    def __init__(self, input_shape=(256, 256, 3), n_qubits=8, n_classes=4, learning_rate=5e-5, base_model_name="MobileNetV2"):
        self.input_shape = input_shape
        self.n_qubits = n_qubits
        self.n_classes = n_classes
        self.learning_rate = learning_rate
        self.base_model_name = base_model_name  # ⭐ เพิ่ม base model parameter

        # Initialize Quantum Device
        self.dev = qml.device('default.qubit', wires=n_qubits)

        # Build the model
        self.model = self._build_model()

    def _qenna_encoding(self, inputs):
        qml.AngleEmbedding(inputs, wires=range(self.n_qubits), rotation='Y')

    def _quantum_convolution(self, params):
        param_idx = 0
        for i in range(0, self.n_qubits - 1, 2):
            qml.RY(params[param_idx], wires=i)
            qml.RY(params[param_idx + 1], wires=i+1)
            qml.CNOT(wires=[i, i+1])
            qml.RZ(params[param_idx + 2], wires=i)
            qml.RZ(params[param_idx + 3], wires=i+1)
            param_idx += 4
        for i in range(self.n_qubits):
            qml.CNOT(wires=[i, (i+1) % self.n_qubits])

    def _quantum_pooling(self):
        return [qml.expval(qml.PauliZ(w)) for w in range(4)]

    def _qenna_circuit(self, inputs, conv1_params, conv2_params):
        self._qenna_encoding(inputs)
        self._quantum_convolution(conv1_params)
        self._quantum_convolution(conv2_params)
        return self._quantum_pooling()

    def _load_base_model(self, input_tensor):
        """Load different CNN backbones based on selection."""
        if self.base_model_name.lower() == "mobilenetv2":
            base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)
        elif self.base_model_name.lower() == "efficientnetv2s":
            base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_tensor=input_tensor)
        else:
            raise ValueError(f"Unknown base model: {self.base_model_name}")

        return base_model

    def _build_model(self):
        # Setup input
        input_img = layers.Input(shape=self.input_shape)

        # Resize to match model requirement
        x = layers.Resizing(224, 224)(input_img)

        # Load base model
        base_model = self._load_base_model(x)
        x = base_model.output  # ⭐ สำคัญ: อย่าลืมใช้ .output

        # Fine-tuning
        for layer in base_model.layers[:80]:
            layer.trainable = False
        for layer in base_model.layers[80:]:
            layer.trainable = True

        # Feature extraction
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dense(8, activation='relu')(x)  # Match 8 qubits

        # Normalize for Quantum
        x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1) * np.pi)(x)

        # Quantum Layer
        weight_shapes = {
            "conv1_params": (16,),
            "conv2_params": (16,)
        }
        qnode = qml.QNode(self._qenna_circuit, self.dev, interface='tf')
        q_layer = KerasLayer(
            qnode,
            weight_shapes=weight_shapes,
            output_dim=4
        )
        x = q_layer(x)

        # Final Dense layers + output
        x = layers.Dense(1024, activation='relu')(x)
        output = layers.Dense(self.n_classes, activation='softmax')(x)

        # Create and compile model
        model = tf.keras.Model(inputs=input_img, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def summary(self):
        return self.model.summary()

    def get_model(self):
        return self.model


In [16]:
model_wrapper = QENNAClassifier(base_model_name="EfficientNetV2S")
model = model_wrapper.get_model()
model.summary()

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resizing_4          │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Resizing)          │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ resizing_4[0][0]  │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 112, 112,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 112, 112,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 112, 112,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 112, 112,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 112, 112,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 112, 112,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 56, 56,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 56, 56,    │        384 │ block2a_expand_c

 Total params: 21,660,556 (82.63 MB)

 Trainable params: 20,541,588 (78.36 MB)

 Non-trainable params: 1,118,968 (4.27 MB)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(xs, ys_encoded, test_size=0.2, random_state=42)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100,
                    batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100


640/640 ━━━━━━━━━━━━━━━━━━━━ 188s 74ms/step - accuracy: 0.4567 - loss: 1.2106 - val_accuracy: 0.5102 - val_loss: 0.9950
Epoch 2/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.5435 - loss: 0.9724 - val_accuracy: 0.6250 - val_loss: 0.8545
Epoch 3/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.6138 - loss: 0.8590 - val_accuracy: 0.6867 - val_loss: 0.7879
Epoch 4/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.6640 - loss: 0.7814 - val_accuracy: 0.7406 - val_loss: 0.6514
Epoch 5/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.7288 - loss: 0.6790 - val_accuracy: 0.8203 - val_loss: 0.5318
Epoch 6/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.7864 - loss: 0.5715 - val_accuracy: 0.8078 - val_loss: 0.5190
Epoch 7/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.8248 - loss: 0.4687 - val_accuracy: 0.8844 - val_loss: 0.3391
Epoch 8/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.8616 - loss: 0.3910 - val